# TM10007 Assignment lipomas G9

In [1]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/Machine-Learning-TM10007-G9/Lipo-MRI.git

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 32.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 823.6/823.6 kB 43.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.1/14.1 MB 31.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.7/731.7 MB 1.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.6/410.6 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 121.6/121.6 MB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.5/56.5 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 124.2/124.2 MB 7.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.0/196.0 MB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.0/166.0 MB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.1/99.1 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 6

# **Data loading**



In [2]:
from worclipo.load_data import load_data

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')

The number of samples: 115
The number of columns: 494


# **Import important functions**

In [3]:
# General packages
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets as ds
import seaborn


# Classifiers
from sklearn import model_selection
from sklearn import metrics
from sklearn import feature_selection
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import svm
from sklearn import decomposition

# **Create X and Y**

In [39]:
X = data.iloc[:, 1:].values
y = np.array(data['label'])

# **Split the data**

In [11]:
# Split data
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, test_size=0.2, stratify=y)

#print(len(X_train))
#print(len(X_test))
#print(len(y_train))
#print(len(y_test))

# **Checks if all the data points are floats**
Is 0.0 a missing value?

In [65]:
for row in X_train:
  for col in row:
    if not isinstance(col, float):
        print(col)

# **Check for missing values**



In [66]:
missing_values = np.sum(np.isnan(X_train))

print("Number of missing values:",missing_values)

Number of missing values: 0


# **Check for duplicates**
Can we assume that duplicates values are duplicates?

In [69]:
# Function to find duplicate columns and delete them
def delete_duplicate_columns(arr):
    num_cols = arr.shape[1]
    duplicate_columns = []
    for i in range(num_cols):
        for j in range(i + 1, num_cols):
            if np.array_equal(arr[:, i], arr[:, j]):
                duplicate_columns.append(j)  # Append index of the duplicate column
    if duplicate_columns:
        # Delete duplicate columns
        arr_without_duplicates = np.delete(arr, duplicate_columns, axis=1)
        return arr_without_duplicates
    else:
        return arr

# Remove duplicate columns
X_train_clean = delete_duplicate_columns(X_train)

print("Number of columns in original array:",X_train.shape[1])
print("Number of columns after removal of duplicate columns:",X_train_clean.shape[1])

Number of columns in original array: 493
Number of columns after removal of duplicate columns: 466


# **Checking for outliers**
Using the Z-score

In [70]:
from scipy import stats

num_rows, num_cols = X_train_clean.shape
total_outliers = 0

for i in range(num_cols):
    z = np.abs(stats.zscore(X_train_clean[:,i]))

    threshold = 3
    outliers = X_train_clean[z > threshold, i]
    total_outliers +=(len(outliers))

print(f'Total number of outliers  = {total_outliers}/{num_rows*num_cols}')
print(f'Avarage number of outliers per feature = {round(total_outliers/num_cols,2)}/{num_rows}')


Total number of outliers  = 628/42872
Avarage number of outliers per feature = 1.35/92


# **Check if the data is normally distributed**
Using the Kolmogorov Smirnov

In [71]:
total_not_normal = 0
p_threshold = 0.05

for l in range(num_cols):
    kstest_result = stats.kstest(X_train_clean[:,l], 'norm')
    if kstest_result.pvalue < p_threshold:
        total_not_normal += 1

print(f'Total features that are not normally distributed = {total_not_normal}/{num_cols}')


Total features that are not normally distributed = 465/466


# **Scaling**
Robust scaling because outliers and non Gaussian distribution

In [73]:
# Scale the data to be normal
scaler = preprocessing.RobustScaler()
scaler.fit(X_train_clean)
X_train_scaled = scaler.transform(X_train_clean)

# **Cross-validation**

In [74]:
# Initialize KFold cross-validator
kf = model_selection.StratifiedKFold(n_splits=5)

# Iterate over each fold
for train_index, val_index in kf.split(X_train_clean, y_train):
    # Split data into train and validation sets for this fold
    X_train_fold, X_val_fold = X_train_scaled[train_index], X_train_scaled[val_index]
    y_train_fold, y_val_fold = y_train[train_index], y_train[val_index]

print(len(X_train_fold))
print(len(X_val_fold))
print(len(y_train_fold))
print(len(y_val_fold))

74
18
74
18


# **Feature selection**